In [9]:
"""
This Weights & Biases sample script trains a basic CNN on the
Fashion-MNIST dataset. It takes black and white images of clothing
and labels them as "pants", "belt", etc. This script is designed
to demonstrate the wandb integration with Keras.
"""

from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import TensorBoard
import random

In [10]:
#Import wandb libraries
import wandb
from wandb.keras import WandbCallback

In [11]:
# Initialize wandb and save hyperparameters
wandb.init(
  project="sample-project",
  config={
    "dropout": 0.2,
    "hidden_layer_size": 128,
    "layer_1_size": 16,
    "layer_2_size": 32,
    "learn_rate": 0.01,
    "decay": 1e-6,
    "momentum": 0.9,
    "epochs": 8 }
)
config = wandb.config

In [12]:
(X_train_orig, y_train_orig), (X_test, y_test) = fashion_mnist.load_data()

In [13]:
# Reducing the dataset size to 10,000 examples for faster train time
true = list(map(lambda x: True if random.random() < 0.167 else False, range(60000)))
ind = []
for i, x in enumerate(true):
    if x == True: ind.append(i)

In [14]:
X_train = X_train_orig[ind, :, :]
y_train = y_train_orig[ind]

In [15]:
img_width=28
img_height=28
labels =["T-shirt/top","Trouser","Pullover","Dress",
    "Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [16]:
X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

In [17]:
#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [22]:
sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

# build model
model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(config.hidden_layer_size, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#Add Keras WandbCallback
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels)])

Epoch 1/5
316/316 [==============================] - 5s 15ms/step - loss: 1.3014 - accuracy: 0.5289 - val_loss: 0.8151 - val_accuracy: 0.7099
Epoch 2/5
316/316 [==============================] - 6s 18ms/step - loss: 0.7789 - accuracy: 0.7061 - val_loss: 0.6942 - val_accuracy: 0.7410
Epoch 3/5
316/316 [==============================] - 6s 20ms/step - loss: 0.7237 - accuracy: 0.7218 - val_loss: 0.6300 - val_accuracy: 0.7350
Epoch 4/5
316/316 [==============================] - 7s 21ms/step - loss: 0.6746 - accuracy: 0.7492 - val_loss: 0.6034 - val_accuracy: 0.7680
Epoch 5/5
316/316 [==============================] - 7s 22ms/step - loss: 0.6592 - accuracy: 0.7473 - val_loss: 0.6403 - val_accuracy: 0.7626


In [21]:
wandb.init(
  project="sample-project",
  config={
    "dropout": 0.4,
    "hidden_layer_size": 256,
    "layer_1_size": 16,
    "layer_2_size": 32,
    "learn_rate": 0.1,
    "decay": 1e-6,
    "momentum": 0.9,
    "epochs": 5 }
)
config = wandb.config

epoch,7
loss,0.36187
accuracy,0.86241
val_loss,0.41482
val_accuracy,0.8485
_runtime,798
_timestamp,1619228416
_step,7
best_val_loss,0.40083
best_epoch,5


epoch,▁▂▃▄▅▆▇█
loss,█▃▂▂▂▁▁▁
accuracy,▁▆▇▇▇███
val_loss,██▄▃▃▁▂▁
val_accuracy,▁▁▅▅▇█▇█
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█
